On rajoute les données meteo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import cm
from matplotlib.colors import ListedColormap

from sklearn.ensemble import GradientBoostingRegressor

In [2]:
stations_debit = pd.read_csv("../Data/Base/Stations_Debit.csv")
liste_stations_debit = list(stations_debit["Code station"])

mesures_train_X = pd.read_csv("../Data/Base/Mesures_Train_X.csv")
mesures_train_X["Date"] = pd.to_datetime(mesures_train_X["Date"], format = "%Y/%m/%d %H:%M:%S")
mesures_train_Y = pd.read_csv("../Data/Base/Mesures_Train_Y.csv")
mesures_train_Y["Date"] = pd.to_datetime(mesures_train_Y["Date"], format = "%Y/%m/%d %H:%M:%S")

train_split_X = pd.read_csv("../Data/Base/Index_CV_X.csv")
train_split_Y = pd.read_csv("../Data/Base/Index_CV_Y.csv")

mesures_test_X = pd.read_csv("../Data/Base/Mesures_Test_X.csv")
mesures_test_X["Date"] = pd.to_datetime(mesures_test_X["Date"], format = "%Y/%m/%d %H:%M:%S")
mesures_test_Y = pd.read_csv("../Data/Base/Mesures_Test_Y.csv")
mesures_test_Y["Date"] = pd.to_datetime(mesures_test_Y["Date"], format = "%Y/%m/%d %H:%M:%S")

In [3]:
stations_meteo = pd.read_csv("../Data/Base/Stations_Meteo.csv")
liste_stations_meteo = list(stations_meteo["ID"].apply(lambda x: str(x).rjust(5, "0")))
colonnes_meteo = ["Pression", "Vent_Nord", "Vent_Est", "Vitesse_vent", "Temperature", "Humidite", "Precipitations"]
tempo = []
for code in liste_stations_meteo:
    tempo += [tp + "_" + code for tp in colonnes_meteo]
colonnes_meteo_stations = tempo

In [4]:
%run Eval_fcts.py
%run Standardize_fcts.py

In [5]:
%run Regression_GAM_Gamma.py

# Val croisée

In [6]:
n_knots = 10
lr_grid = [1, 3, 10, 30, 100, 300, 1000]
max_lag = 7

In [ ]:
for curr_lr in lr_grid:
    for curr_lag in tqdm(range(0,max_lag+1)):
        cv_scores_RMSE_standard = pd.DataFrame({"Code station": liste_stations_debit})
        cv_scores_MAE_standard = pd.DataFrame({"Code station": liste_stations_debit})
        cv_scores_R2_standard = pd.DataFrame({"Code station": liste_stations_debit})

        cv_scores_RMSE = pd.DataFrame({"Code station": liste_stations_debit})
        cv_scores_MAE = pd.DataFrame({"Code station": liste_stations_debit})
        cv_scores_R2 = pd.DataFrame({"Code station": liste_stations_debit})

        for curr_split in range(9):

            # Donnees
            curr_train_X = mesures_train_X[train_split_X["Train_" + str(curr_split)]]
            curr_train_Y = mesures_train_Y[train_split_Y["Train_" + str(curr_split)]]
            curr_test_X = mesures_train_X[train_split_X["Test_" + str(curr_split)]]
            curr_test_Y = mesures_train_Y[train_split_Y["Test_" + str(curr_split)]]
            liste_dates = curr_test_Y["Date"]

            curr_train_X_mean = curr_train_X[liste_stations_debit + colonnes_meteo_stations].mean()
            for code in liste_stations_debit:
                curr_train_X_mean[code] = 0
            curr_train_X_std = curr_train_X[liste_stations_debit + colonnes_meteo_stations].std()
            curr_train_X_standard = fct_Standardize(curr_train_X, curr_train_X_mean,
                                                    curr_train_X_std, liste_stations_debit + colonnes_meteo_stations)
            curr_train_X_standard["Date"] = curr_train_X["Date"]
            #GAM
            spline_fit = fct_Regression_SplineGamma_fit(curr_train_X_standard, liste_stations_debit, n_knots)
            curr_train_X_standard_residus = fct_Regression_SplineGamma_residus(spline_fit, 
                                                                               curr_train_X_standard, 
                                                                               liste_stations_debit)
            curr_train_X_standard_residus = pd.concat([curr_train_X_standard_residus, 
                                                          curr_train_X_standard[colonnes_meteo_stations]],
                                                         axis = 1)
            
             # Ajout AR
            colonnes_reg = colonnes_meteo_stations
            for i in range(curr_lag+1):
                variable = curr_train_X_standard_residus[liste_stations_debit].shift(i+7)
                variable.columns = [code+"_"+str(i) for code in liste_stations_debit]
                curr_train_X_standard_residus = pd.concat([curr_train_X_standard_residus, variable], axis = 1)
                colonnes_reg = colonnes_reg + [code+"_"+str(i) for code in liste_stations_debit]
            curr_train_X_standard_residus = curr_train_X_standard_residus[(7+curr_lag):]
            
            curr_test_X_standard =fct_Standardize(curr_test_X, curr_train_X_mean,
                                                  curr_train_X_std, liste_stations_debit + colonnes_meteo_stations)
            curr_test_X_standard["Date"] = curr_test_X["Date"].values
            curr_test_X_standard_residus = fct_Regression_SplineGamma_residus(spline_fit,
                                                                              curr_test_X_standard, 
                                                                              liste_stations_debit)
            curr_test_X_standard_residus = pd.concat([curr_test_X_standard_residus, 
                                                      curr_test_X_standard[colonnes_meteo_stations]],
                                                     axis = 1)
            # Ajout variables AR
            for i in range(curr_lag+1):
                variable = curr_test_X_standard_residus[liste_stations_debit].shift(i+7)
                variable.columns = [code+"_"+str(i) for code in liste_stations_debit]
                curr_test_X_standard_residus = pd.concat([curr_test_X_standard_residus, variable], axis = 1)
            curr_test_X_standard_residus = curr_test_X_standard_residus[(7+curr_lag):]
            
            curr_test_Y_standard = fct_Standardize(curr_test_Y, curr_train_X_mean,
                                                    curr_train_X_std, liste_stations_debit)
            curr_test_Y_standard["Date"] = curr_test_Y["Date"]

            curr_RMSE_standard = []
            curr_MAE_standard = []
            curr_R2_standard = []
            curr_RMSE = []
            curr_MAE = []
            curr_R2 = []
            
            for code in liste_stations_debit:

                # Entrainement
                model = GradientBoostingRegressor(learning_rate = curr_lr/1000)
                model.fit(X = curr_train_X_standard_residus[colonnes_reg],
                      y = curr_train_X_standard_residus[code])

                # Predictions Standard
                predictions_Y_standard_residus = model.predict(curr_test_X_standard_residus[colonnes_reg])
                predictions_Y_standard_residus = pd.DataFrame(predictions_Y_standard_residus, columns=[code])
                predictions_Y_standard_residus["Date"] = curr_test_X_standard_residus["Date"].values
                # Filtre sur les dates
                resultat = pd.DataFrame()
                for curr_date in liste_dates:
                    resultat = pd.concat([resultat, predictions_Y_standard_residus[predictions_Y_standard_residus["Date"] == curr_date]])
                resultat = resultat.sort_values(by = "Date")
                predictions_Y_standard_residus = resultat.copy()
                # On rajoute la compo saisonnalité
                predictions_Y_standard_saisonnalite = fct_Regression_SplineGamma_predict(spline_fit, liste_dates, [code])
                predictions_Y_standard = predictions_Y_standard_residus[["Date"]].copy()
                predictions_Y_standard[code] = predictions_Y_standard_residus[code].values + predictions_Y_standard_saisonnalite[code].values

                # Scores standards
                curr_RMSE_standard.append(fct_RMSE(curr_test_Y_standard, predictions_Y_standard, [code])["RMSE"][0])
                curr_MAE_standard.append(fct_MAE(curr_test_Y_standard, predictions_Y_standard, [code])["MAE"][0])
                curr_R2_standard.append(fct_R2(curr_test_Y_standard, predictions_Y_standard, [code])["R2"][0])
                # Score
                predictions_Y = fct_StandardizeInverse(predictions_Y_standard, 
                                                       curr_train_X_mean, curr_train_X_std, 
                                                       [code])
                predictions_Y["Date"] = predictions_Y_standard["Date"].values
                curr_RMSE.append(fct_RMSE(curr_test_Y, predictions_Y, [code])["RMSE"][0])
                curr_MAE.append(fct_MAE(curr_test_Y, predictions_Y, [code])["MAE"][0])
                curr_R2.append(fct_R2(curr_test_Y, predictions_Y, [code])["R2"][0])

            # On rassemble
            cv_scores_RMSE_standard["Split_" + str(curr_split)] = curr_RMSE_standard
            cv_scores_MAE_standard["Split_" + str(curr_split)] = curr_MAE_standard
            cv_scores_R2_standard["Split_" + str(curr_split)] = curr_R2_standard

            cv_scores_RMSE["Split_" + str(curr_split)] = curr_RMSE
            cv_scores_MAE["Split_" + str(curr_split)] = curr_MAE
            cv_scores_R2["Split_" + str(curr_split)] = curr_R2

        # Calcul des scores moyens du split
        cv_moyen_RMSE_standard = []
        cv_moyen_MAE_standard = []
        cv_moyen_R2_standard = []
        cv_moyen_RMSE = []
        cv_moyen_MAE = []
        cv_moyen_R2 = []

        for code in liste_stations_debit:
            score_RMSE = np.mean(cv_scores_RMSE_standard[cv_scores_RMSE_standard["Code station"] == code][["Split_" + str(i) for i in range(9)]].iloc[0,:])
            cv_moyen_RMSE_standard.append(score_RMSE)
            score_MAE = np.mean(cv_scores_MAE_standard[cv_scores_MAE_standard["Code station"] == code][["Split_" + str(i) for i in range(9)]].iloc[0,:])
            cv_moyen_MAE_standard.append(score_MAE)
            score_R2 = np.mean(cv_scores_R2_standard[cv_scores_R2_standard["Code station"] == code][["Split_" + str(i) for i in range(9)]].iloc[0,:])
            cv_moyen_R2_standard.append(score_R2)
            score_RMSE = np.mean(cv_scores_RMSE[cv_scores_RMSE["Code station"] == code][["Split_" + str(i) for i in range(9)]].iloc[0,:])
            cv_moyen_RMSE.append(score_RMSE)
            score_MAE = np.mean(cv_scores_MAE[cv_scores_MAE["Code station"] == code][["Split_" + str(i) for i in range(9)]].iloc[0,:])
            cv_moyen_MAE.append(score_MAE)
            score_R2 = np.mean(cv_scores_R2[cv_scores_R2["Code station"] == code][["Split_" + str(i) for i in range(9)]].iloc[0,:])
            cv_moyen_R2.append(score_R2)

        cv_scores_RMSE_standard["Moyenne"] = cv_moyen_RMSE_standard
        cv_scores_RMSE_standard.to_csv("../Data/GAMGradientBoostingAR/CV_RMSE_standard_" + str(curr_lr) + "_" + str(curr_lag) + ".csv",
                              index=False)
        cv_scores_MAE_standard["Moyenne"] = cv_moyen_MAE_standard
        cv_scores_MAE_standard.to_csv("../Data/GAMGradientBoostingAR/CV_MAE_standard_" + str(curr_lr) + "_" + str(curr_lag) + ".csv",
                              index=False)
        cv_scores_R2_standard["Moyenne"] = cv_moyen_R2_standard
        cv_scores_R2_standard.to_csv("../Data/GAMGradientBoostingAR/CV_R2_standard_" + str(curr_lr) + "_" + str(curr_lag) + ".csv",
                              index=False)
        cv_scores_RMSE["Moyenne"] = cv_moyen_RMSE
        cv_scores_RMSE.to_csv("../Data/GAMGradientBoostingAR/CV_RMSE_" + str(curr_lr) + "_" + str(curr_lag) + ".csv",
                              index=False)
        cv_scores_MAE["Moyenne"] = cv_moyen_MAE
        cv_scores_MAE.to_csv("../Data/GAMGradientBoostingAR/CV_MAE_" + str(curr_lr) + "_" + str(curr_lag) + ".csv",
                              index=False)
        cv_scores_R2["Moyenne"] = cv_moyen_R2
        cv_scores_R2.to_csv("../Data/GAMGradientBoostingAR/CV_R2_" + str(curr_lr) + "_" + str(curr_lag) + ".csv",
                              index=False)


 38%|██████████████▋                        | 3/8 [2:41:58<4:29:04, 3228.86s/it]

In [ ]:
colonnes_meteo_stations

## Selection

In [ ]:
n_cours_eau = 3
cours_eau = list(np.unique(stations_debit["Cours eau"]))
cours_eau_cmap = cm.get_cmap(ListedColormap(["red", "green", "blue"]))
cours_eau_couleur = pd.DataFrame({"Cours eau": cours_eau, "Index": range(n_cours_eau), "Couleur": ["red", "green", "blue"]})
cours_eau_couleur

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lr in lr_grid:
    curr_moyen = {}
    cv_scores = pd.read_csv("../Data/GradientBoosting_Meteo/CV_RMSE_lr_" + str(curr_lr) + ".csv")
    for code in liste_stations_debit:
        curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
    curr_moyen = pd.DataFrame(curr_moyen)
    curr_moyen["lr"] = curr_lr/1000
    cv_moyen = pd.concat([cv_moyen, curr_moyen])

fig, axs = plt.subplots(n_cours_eau, 1, figsize = (15,5))
for i in range(n_cours_eau):
    stations = list(stations_debit[stations_debit["Cours eau"] == list(cours_eau_couleur[cours_eau_couleur["Index"] == i]["Cours eau"])[0]]["Code station"])
    for code in stations:
        axs[i].plot(cv_moyen["lr"], cv_moyen[code],
                   color = cours_eau_cmap(i))
        axs[i].set_xscale("log")
plt.show()

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lr in lr_grid:
    curr_moyen = {}
    cv_scores = pd.read_csv("../Data/GradientBoosting_Meteo/CV_MAE_lr_" + str(curr_lr) + ".csv")
    for code in liste_stations_debit:
        curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
    curr_moyen = pd.DataFrame(curr_moyen)
    curr_moyen["lr"] = curr_lr/1000
    cv_moyen = pd.concat([cv_moyen, curr_moyen])

fig, axs = plt.subplots(n_cours_eau, 1, figsize = (15,5))
for i in range(n_cours_eau):
    stations = list(stations_debit[stations_debit["Cours eau"] == list(cours_eau_couleur[cours_eau_couleur["Index"] == i]["Cours eau"])[0]]["Code station"])
    for code in stations:
        axs[i].plot(cv_moyen["lr"], cv_moyen[code],
                   color = cours_eau_cmap(i))
        axs[i].set_xscale("log")
plt.show()

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lr in lr_grid:
    curr_moyen = {}
    cv_scores = pd.read_csv("../Data/GradientBoosting_Meteo/CV_R2_lr_" + str(curr_lr) + ".csv")
    for code in liste_stations_debit:
        curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
    curr_moyen = pd.DataFrame(curr_moyen)
    curr_moyen["lr"] = curr_lr/1000
    cv_moyen = pd.concat([cv_moyen, curr_moyen])

fig, axs = plt.subplots(n_cours_eau, 1, figsize = (15,5))
for i in range(n_cours_eau):
    stations = list(stations_debit[stations_debit["Cours eau"] == list(cours_eau_couleur[cours_eau_couleur["Index"] == i]["Cours eau"])[0]]["Code station"])
    for code in stations:
        axs[i].plot(cv_moyen["lr"], cv_moyen[code],
                   color = cours_eau_cmap(i))
        axs[i].set_xscale("log")
plt.show()

## Standard

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lr in lr_grid:
    curr_moyen = {}
    cv_scores = pd.read_csv("../Data/GradientBoosting_Meteo/CV_RMSE_standard_lr_" + str(curr_lr) + ".csv")
    for code in liste_stations_debit:
        curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
    curr_moyen = pd.DataFrame(curr_moyen)
    curr_moyen["lr"] = curr_lr/1000
 + ".csv")    cv_moyen = pd.concat([cv_moyen, curr_moyen])

fig, axs = plt.subplots(n_cours_eau, 1, figsize = (15,5))
for i in range(n_cours_eau):
    stations = list(stations_debit[stations_debit["Cours eau"] == list(cours_eau_couleur[cours_eau_couleur["Index"] == i]["Cours eau"])[0]]["Code station"])
    for code in stations:
        axs[i].plot(cv_moyen["lr"], cv_moyen[code],
                   color = cours_eau_cmap(i))
        axs[i].set_xscale("log")
plt.show()

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lr in lr_grid:
    curr_moyen = {}
    cv_scores = pd.read_csv("../Data/GradientBoosting_Meteo/CV_MAE_standard_lr_" + str(curr_lr) + ".csv")
    for code in liste_stations_debit:
        curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
    curr_moyen = pd.DataFrame(curr_moyen)
    curr_moyen["lr"] = curr_lr/1000
    cv_moyen = pd.concat([cv_moyen, curr_moyen])

fig, axs = plt.subplots(n_cours_eau, 1, figsize = (15,5))
for i in range(n_cours_eau):
    stations = list(stations_debit[stations_debit["Cours eau"] == list(cours_eau_couleur[cours_eau_couleur["Index"] == i]["Cours eau"])[0]]["Code station"])
    for code in stations:
        axs[i].plot(cv_moyen["lr"], cv_moyen[code],
                   color = cours_eau_cmap(i))
        axs[i].set_xscale("log")
plt.show()

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lr in lr_grid:
    curr_moyen = {}
    cv_scores = pd.read_csv("../Data/GradientBoosting_Meteo/CV_R2_standard_lr_" + str(curr_lr) + ".csv")
    for code in liste_stations_debit:
        curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
    curr_moyen = pd.DataFrame(curr_moyen)
    curr_moyen["lr"] = curr_lr/1000
    cv_moyen = pd.concat([cv_moyen, curr_moyen])

fig, axs = plt.subplots(n_cours_eau, 1, figsize = (15,5))
for i in range(n_cours_eau):
    stations = list(stations_debit[stations_debit["Cours eau"] == list(cours_eau_couleur[cours_eau_couleur["Index"] == i]["Cours eau"])[0]]["Code station"])
    for code in stations:
        axs[i].plot(cv_moyen["lr"], cv_moyen[code],
                   color = cours_eau_cmap(i))
        axs[i].set_xscale("log")
plt.show()

## Moyen

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lag in range(max_lag+1):
    for curr_lr in lr_grid:
        curr_moyen = {} 
        cv_scores = pd.read_csv("../Data/GAMGradientBoostingAR/CV_RMSE_standard_" + str(curr_lr) + "_" + str(curr_lag) + ".csv")
        for code in liste_stations_debit:
            curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
        curr_moyen = pd.DataFrame(curr_moyen)
        curr_moyen["lr"] = curr_lr/1000
        curr_moyen["Lags"] = curr_lag
        cv_moyen = pd.concat([cv_moyen, curr_moyen])

In [ ]:
plt.plot(cv_moyen.groupby("lr").mean().index, cv_moyen.groupby("lr").mean()[liste_stations_debit].mean(axis = 1).values)
plt.xscale("log")
plt.savefig("../Data/Figures/GAMBoostAR/GAMBoostAR_RMSE_lr.png")

In [ ]:
plt.plot(cv_moyen.groupby("Lags").mean().index, cv_moyen.groupby("Lags").mean()[liste_stations_debit].mean(axis = 1).values)
plt.savefig("../Data/Figures/GAMBoostAR/GAMBoostAR_RMSE_AR.png")

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lag in range(max_lag+1):
    for curr_lr in lr_grid:
        curr_moyen = {} 
        cv_scores = pd.read_csv("../Data/GAMGradientBoostingAR/CV_MAE_standard_" + str(curr_lr) + "_" + str(curr_lag) + ".csv")
        for code in liste_stations_debit:
            curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
        curr_moyen = pd.DataFrame(curr_moyen)
        curr_moyen["lr"] = curr_lr/1000
        curr_moyen["Lags"] = curr_lag
        cv_moyen = pd.concat([cv_moyen, curr_moyen])

In [ ]:
plt.plot(cv_moyen.groupby("lr").mean().index, cv_moyen.groupby("lr").mean()[liste_stations_debit].mean(axis = 1).values)
plt.xscale("log")
plt.savefig("../Data/Figures/GAMBoostAR/GAMBoostAR_MAE_lr.png")

In [ ]:
plt.plot(cv_moyen.groupby("Lags").mean().index, cv_moyen.groupby("Lags").mean()[liste_stations_debit].mean(axis = 1).values)
plt.savefig("../Data/Figures/GAMBoostAR/GAMBoostAR_MAE_AR.png")

In [ ]:
cv_moyen = pd.DataFrame()

for curr_lag in range(max_lag+1):
    for curr_lr in lr_grid:
        curr_moyen = {} 
        cv_scores = pd.read_csv("../Data/GAMGradientBoostingAR/CV_R2_standard_" + str(curr_lr) + "_" + str(curr_lag) + ".csv")
        for code in liste_stations_debit:
            curr_moyen[code] = list(cv_scores[cv_scores["Code station"] == code]["Moyenne"])
        curr_moyen = pd.DataFrame(curr_moyen)
        curr_moyen["lr"] = curr_lr/1000
        curr_moyen["Lags"] = curr_lag
        cv_moyen = pd.concat([cv_moyen, curr_moyen])

In [ ]:
plt.plot(cv_moyen.groupby("lr").mean().index, cv_moyen.groupby("lr").mean()[liste_stations_debit].mean(axis = 1).values)
plt.xscale("log")
plt.savefig("../Data/Figures/GAMBoostAR/GAMBoostAR_R2_lr.png")

In [ ]:
plt.plot(cv_moyen.groupby("Lags").mean().index, cv_moyen.groupby("Lags").mean()[liste_stations_debit].mean(axis = 1).values)
plt.savefig("../Data/Figures/GAMBoostAR/GAMBoostAR_R2_AR.png")

In [ ]:
pd.read_csv("../Data/GAMGradientBoostingAR/CV_RMSE_standard_30_0.csv")["Moyenne"].mean()

In [ ]:
pd.read_csv("../Data/GAMGradientBoostingAR/CV_MAE_standard_30_0.csv")["Moyenne"].mean()

In [ ]:
pd.read_csv("../Data/GAMGradientBoostingAR/CV_R2_standard_30_0.csv")["Moyenne"].mean()

# Test

In [ ]:
lr = 30

In [ ]:
# Donnees
mesures_train_X_mean = mesures_train_X[liste_stations_debit + colonnes_meteo_stations].mean()
mesures_train_X_std = mesures_train_X[liste_stations_debit + colonnes_meteo_stations].std()
mesures_train_X_standard = fct_Standardize(mesures_train_X, 
                                           mesures_train_X_mean, mesures_train_X_std, 
                                           liste_stations_debit + colonnes_meteo_stations)
mesures_train_X_standard["Date"] = mesures_train_X["Date"]

mesures_test_X_standard =fct_Standardize(mesures_test_X, mesures_train_X_mean, mesures_train_X_std, 
                                         liste_stations_debit + colonnes_meteo_stations)
mesures_test_X_standard["Date"] = mesures_test_X["Date"].values
mesures_test_Y_standard = fct_Standardize(mesures_test_Y, mesures_train_X_mean, mesures_train_X_std, 
                                          liste_stations_debit)
mesures_test_Y_standard["Date"] = mesures_test_Y["Date"]
liste_dates = mesures_test_Y["Date"]

In [ ]:
scores_RMSE_standard = []
scores_MAE_standard = []
scores_R2_standard = []
scores_RMSE = []
scores_MAE = []
scores_R2 = []

predictions_test_Y_standard = mesures_test_Y[["Date"]]
predictions_test_Y = mesures_test_Y[["Date"]]

for code in tqdm(liste_stations_debit):
        
        
        # Entrainement
        model = GradientBoostingRegressor(learning_rate = lr/1000)
        model.fit(X = mesures_train_X_standard[colonnes_meteo_stations],
                  y = mesures_train_X_standard[code])
        
        # Predictions Standard
        predictions_Y_standard = model.predict(mesures_test_X_standard[colonnes_meteo_stations])
        predictions_Y_standard = pd.DataFrame(predictions_Y_standard, columns=[code])
        predictions_Y_standard["Date"] = mesures_test_X_standard["Date"].values
        # Filtre sur les dates
        resultat = pd.DataFrame()
        for curr_date in liste_dates:
            resultat = pd.concat([resultat, predictions_Y_standard[predictions_Y_standard["Date"] == curr_date]])
        resultat = resultat.sort_values(by = "Date")
        predictions_Y_standard = resultat.copy()
        predictions_test_Y_standard = predictions_test_Y_standard.merge(predictions_Y_standard)
        
        predictions_Y = fct_StandardizeInverse(predictions_Y_standard, 
                                                   mesures_train_X_mean, mesures_train_X_std, 
                                                   [code])
        predictions_Y["Date"] = predictions_Y_standard["Date"].values
        predictions_test_Y = predictions_test_Y.merge(predictions_Y)
        
        # Scores standards
        scores_RMSE_standard.append(fct_RMSE(mesures_test_Y_standard, predictions_Y_standard, [code])["RMSE"][0])
        scores_MAE_standard.append(fct_MAE(mesures_test_Y_standard, predictions_Y_standard, [code])["MAE"][0])
        scores_R2_standard.append(fct_R2(mesures_test_Y_standard, predictions_Y_standard, [code])["R2"][0])
        # Score
        scores_RMSE.append(fct_RMSE(mesures_test_Y, predictions_Y, [code])["RMSE"][0])
        scores_MAE.append(fct_MAE(mesures_test_Y, predictions_Y, [code])["MAE"][0])
        scores_R2.append(fct_R2(mesures_test_Y, predictions_Y, [code])["R2"][0])        

In [ ]:
test_scores = pd.DataFrame({"Code station": liste_stations_debit,
                            "RMSE": scores_RMSE,
                            "MAE": scores_MAE,
                            "R2": scores_R2})
test_scores.to_csv("../Data/GAMGradientBoostingAR/Test_scores.csv")
test_scores

In [ ]:
test_scores_standard = pd.DataFrame({"Code station": liste_stations_debit,
                            "RMSE": scores_RMSE,
                            "MAE": scores_MAE,
                            "R2": scores_R2})
test_scores_standard.to_csv("../Data/GAMGradientBoostingAR/Test_scores_standard.csv")
test_scores_standard

In [ ]:
test_scores_standard[["RMSE", "MAE", "R2"]].mean()

In [ ]:
n_stations = len(liste_stations_debit)
fig, axs = plt.subplots(n_stations, 1, figsize = (15,30), sharex=True)
for i in range(n_stations):
    code = liste_stations_debit[i]
    axs[i].plot(mesures_test_Y_standard[code], label = "True")
    axs[i].plot(predictions_test_Y_standard[code], label = "Predictions")
    axs[i].legend()

In [ ]:
n_stations = len(liste_stations_debit)
fig, axs = plt.subplots(n_stations, 1, figsize = (15,30), sharex=True)
for i in range(n_stations):
    code = liste_stations_debit[i]
    axs[i].plot(mesures_test_Y[code], label = "True")
    axs[i].plot(predictions_test_Y[code], label = "Predictions")
    axs[i].legend()

In [ ]:
from sklearn.inspection import permutation_importance
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0])[0:20] + 0.5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx][0:20], align="center")
plt.yticks(pos, np.array(colonnes_meteo_stations)[sorted_idx][0:20])
plt.title("Feature Importance (MDI)")

print(feature_importance)

result = permutation_importance(
    model, mesures_test_X_standard[colonnes_meteo_stations], mesures_test_X_standard[code], n_repeats=10, random_state=42, n_jobs=20
)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(
    result.importances[sorted_idx][0:20].T,
    vert=False,
    labels=np.array(mesures_train_X.columns[1:])[sorted_idx][0:20],
)
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()